In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB

In [40]:
# Imports features
data = pd.read_csv("data/features.csv")
data = data.drop(['id'], axis = 1)

In [41]:
data.head()

,dump_line,download_file,open_file,connect_socket,impersonate_user,classnum
0,0,0,15,0,0,8
1,3434,0,241,12,12,6
2,0,0,0,0,0,12
3,0,0,21,0,0,8
4,636,0,12,2,0,10


In [35]:
def calcAccuracy(pred, obs):
    assert (pred.shape[0] == obs.shape[0])
    correct = 0
    for i in xrange(pred.shape[0]):
        if pred[i] == obs[i]:
            correct += 1
    return 100.0 * correct / pred.shape[0]

In [45]:
def crossValidation(n):
    # Determines size of blocks
    dataSize = data.shape[0]
    blockSize = dataSize / n
    
    # Percent correct
    accuracy = []
    
    for i in xrange(n):
        # Calculate start and end indices of validation data
        start = i * blockSize
        end = start + blockSize
        
        # Training and test data
        featsTrain = pd.concat((data.iloc[0:start], data.iloc[end:dataSize]), axis=0)
        featsTest = data.iloc[start:end]
        
        # Store classes
        classTrain = featsTrain.classnum.values
        featsTrain = featsTrain.drop(['classnum'], axis = 1)
        classTest = featsTest.classnum.values
        featsTest = featsTest.drop(['classnum'], axis = 1)
        
        # Training / testing values
        valsTrain = featsTrain.values
        valsTest = featsTest.values
        
        # Classifier goes here
        C = GaussianNB()
        C.fit(valsTrain, classTrain)
        classPred = C.predict(valsTest)
        
        # Calculate accuracy
        accuracy.append(calcAccuracy(classPred, classTest))
        
    print accuracy
        
    print "Mean: " + str(np.mean(accuracy))
    print "Variance: " + str(np.var(accuracy))
    print "Std Dev: " + str(np.std(accuracy))

In [46]:
crossValidation(10)

[14.61038961038961, 18.506493506493506, 14.935064935064934, 11.688311688311689, 13.311688311688311, 10.38961038961039, 13.636363636363637, 14.61038961038961, 14.935064935064934, 19.48051948051948]
Mean: 14.6103896104
Variance: 6.80974869287
Std Dev: 2.60954951914
